In [1]:
# Necessary libraries
import pandas as pd
import sqlite3
import datetime


In [2]:
# Connecting to the SQLite database file
conn = sqlite3.connect(r'C:\Users\klaud\Desktop\Code\Data\OctoPrint_API1.db')
# Function that fetches all sqlite tables
def sql_fetch(conn):
    c = conn.cursor()
    c.execute('SELECT name from sqlite_master where type= "table"')

    print(c.fetchall())

sql_fetch(conn)

[('P_1',), ('P_delays_1',), ('P_2',), ('P_delays_2',), ('P_3',), ('P_delays_3',)]


# Selecting first dataset and cleaning it 

In [3]:
#Data From first Print
df1 = pd.read_sql_query("SELECT * FROM P_1", conn)
df1_delays = pd.read_sql_query("SELECT * FROM P_delays_1", conn)
#Merging delays and print data, without selecting date as it will be duplicate
df1 = pd.concat([df1, df1_delays[['timetime','timeperf_counter']]], axis=1)

In [4]:
df1

,Build_platform_temperature,Build_platform_target_temperature,Hot_end_temperature,Hot_end_temperature_target,Progress,User,Object_,Process_time,Fan_speed,Feed_rate,Current_height,Average_layer_duration,Last_layer_duration,Current_layer,Total_layers,State,Date,timetime,timeperf_counter
0,56.3,60.0,172.55,200.0,0.0142198601372001,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,1,-,-,-,-,-,-,105,printing,2021-05-11 13:49:52,0.282866,0.285265
1,56.1,60.0,173.1,200.0,0.0142198601372001,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,1,-,-,-,-,-,-,105,printing,2021-05-11 13:49:53,0.322404,0.322866
2,56.2,60.0,174.03,200.0,0.0142198601372001,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,1,-,-,-,-,-,-,105,printing,2021-05-11 13:49:54,0.300879,0.304485
3,56.2,60.0,175.67,200.0,0.0142198601372001,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,1,-,-,-,-,-,-,105,printing,2021-05-11 13:49:56,0.326536,0.329532
4,56.4,60.0,179.44,200.0,0.0142198601372001,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,1,-,-,-,-,-,-,105,printing,2021-05-11 13:49:57,0.325450,0.328598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,60.0,0.0,199.7,0.0,99.8555280040606,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10462,Off,3000,51.60,72,78,105,105,printing,2021-05-11 16:44:10,0.312674,0.312726
7881,60.0,0.0,199.7,0.0,99.8555280040606,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10462,Off,3000,51.60,72,78,105,105,printing,2021-05-11 16:44:12,0.311074,0.311510
7882,60.0,0.0,199.7,0.0,99.8555280040606,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10462,Off,3000,51.60,72,78,105,105,printing,2021-05-11 16:44:13,0.296630,0.297313
7883,60.0,0.0,199.7,0.0,99.8555280040606,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10462,Off,3000,51.60,72,78,105,105,printing,2021-05-11 16:44:14,0.305507,0.306093


## Identifying the process start and end (removing data that are not the actual process)

In [5]:
#We can see that there are some data where the process has not started
#Such as "Current_layer" is "-" 
#Therefore selecting rows only from first layer
df1 = df1.iloc[df1.Current_layer.values.searchsorted('1', side='left'):]

#Identifying when process was finished
# We can see that last rows of target temperature of build platform and hot end is 0
df1 = df1.astype({'Hot_end_temperature_target': float})
df1 = df1[df1['Hot_end_temperature_target'] > 0]

In [6]:
df1

,Build_platform_temperature,Build_platform_target_temperature,Hot_end_temperature,Hot_end_temperature_target,Progress,User,Object_,Process_time,Fan_speed,Feed_rate,Current_height,Average_layer_duration,Last_layer_duration,Current_layer,Total_layers,State,Date,timetime,timeperf_counter
59,60.9,60.0,199.5,200.0,0.0258691499465678,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,83,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 13:51:10,0.340936,0.341637
60,60.9,60.0,199.5,200.0,0.0258691499465678,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,83,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 13:51:12,0.314106,0.314553
61,60.9,60.0,199.5,200.0,0.0258691499465678,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,83,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 13:51:13,0.322601,0.323360
62,60.9,60.0,199.5,200.0,0.0267309596518527,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,87,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 13:51:14,0.315502,0.315912
63,60.8,60.0,200.2,200.0,0.0280236742097799,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,88,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 13:51:16,0.330048,0.330387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7875,60.0,60.0,200.3,200.0,99.8058996520666,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10456,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 16:44:04,0.404462,0.404947
7876,60.0,60.0,200.3,200.0,99.8195251606829,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10457,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 16:44:05,0.321114,0.321189
7877,60.0,60.0,199.7,200.0,99.8295251250218,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10459,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 16:44:06,0.302241,0.302431
7878,60.0,60.0,199.7,200.0,99.8442353251638,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10460,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 16:44:08,0.294347,0.294783


In [7]:
# We can additionally see the last rows of target temperature of both build platform and hot end is 0 
# therefore removing them 
df1 = df1.astype({'Hot_end_temperature_target': float})
df1 = df1[df1['Hot_end_temperature_target'] > 0]

# Now we can see that "Fan_speed" is "-" when it has not started, but it says "Off" at the end of the process
# Dropping when "Fan_speed" when "Off"
df_filtered1 = df1.loc[df1['Fan_speed'] != 'Off']



### Selecting Data from second and third print and performing the same steps. However, each dataset was looked through separately, in order to confirm that identifying of process start and end is the same for all datasets

In [8]:
# Data From second Print
df2 = pd.read_sql_query("SELECT * FROM P_2", conn)
df2_delays = pd.read_sql_query("SELECT * FROM P_delays_2", conn)
df2 = pd.concat([df2, df2_delays[['timetime','timeperf_counter']]], axis=1)
df2 = df2.iloc[df2.Current_layer.values.searchsorted('1', side='left'):]
df2 = df2.astype({'Hot_end_temperature_target': float})
df2 = df2[df2['Hot_end_temperature_target'] > 0]

df_filtered2 = df2.loc[df2['Fan_speed'] != 'Off']

# Selecting Data from 3rd print and performing same steps

In [9]:
# Data From third Print
df3 = pd.read_sql_query("SELECT * FROM P_3", conn)
df3_delays = pd.read_sql_query("SELECT * FROM P_delays_3", conn)
#Merging delays and print data, without selecting date as it will be duplicate
df3 = pd.concat([df3, df3_delays[['timetime','timeperf_counter']]], axis=1)
#df3

In [10]:
df3 = df3.iloc[df3.Current_layer.values.searchsorted('1', side='left'):]
df3 = df3.astype({'Hot_end_temperature_target': float})
df_filtered3 = df3[df3['Hot_end_temperature_target'] > 0]

#Here we can see that "Fan_speed" did not contain state "Off"
df_filtered3

,Build_platform_temperature,Build_platform_target_temperature,Hot_end_temperature,Hot_end_temperature_target,Progress,User,Object_,Process_time,Fan_speed,Feed_rate,Current_height,Average_layer_duration,Last_layer_duration,Current_layer,Total_layers,State,Date,timetime,timeperf_counter
105,61.5,60.0,200.6,200.0,0.027010569521428,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,146,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 19:57:01,0.311985,0.312525
106,61.5,60.0,200.6,200.0,0.0274270320470888,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,148,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 19:57:03,0.310653,0.311416
107,61.4,60.0,200.6,200.0,0.0278434945727496,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,150,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 19:57:04,0.299598,0.300209
108,61.4,60.0,200.6,200.0,0.0282599570984104,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,151,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 19:57:05,0.311588,0.312159
109,61.4,60.0,200.6,200.0,0.0299704281859458,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,152,-,1200.000,0.20,-,0,1,105,printing,2021-05-11 19:57:07,0.301903,0.302511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7899,59.9,60.0,200.5,200.0,99.7981049170243,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10490,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 22:49:24,0.311641,0.312209
7900,60.0,60.0,200.0,200.0,99.8136032724436,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10491,100%,10800.000,21.00,72,78,105,105,printing,2021-05-11 22:49:26,0.362199,0.362727
7901,60.0,60.0,200.0,200.0,99.823211657857,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10493,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 22:49:27,0.301651,0.301726
7902,60.0,60.0,200.0,200.0,99.8369251738805,_api,Leirmo_Exp1_Main_Artifact_0.2mm_PLA_MK3S_2h50m...,10494,100%,3000.000,21.00,72,78,105,105,printing,2021-05-11 22:49:29,0.333385,0.333882


# Merging datasets from each print

In [11]:
#Adding column which identifies part number
df_filtered1 = df_filtered1.assign(Part = 1)
df_filtered2 = df_filtered2.assign(Part = 2)
df_filtered3 = df_filtered3.assign(Part = 3)

#Merge data frames and generate new indexes 
data_frames = [df_filtered1, df_filtered2, df_filtered3]
df = pd.concat(data_frames,ignore_index=True)


In [12]:
#Checking for missing values "NaN"
df.isna().any()

Build_platform_temperature           False
Build_platform_target_temperature    False
Hot_end_temperature                  False
Hot_end_temperature_target           False
Progress                             False
User                                 False
Object_                              False
Process_time                         False
Fan_speed                            False
Feed_rate                            False
Current_height                       False
Average_layer_duration               False
Last_layer_duration                  False
Current_layer                        False
Total_layers                         False
State                                False
Date                                 False
timetime                             False
timeperf_counter                     False
Part                                 False
dtype: bool

In [13]:
#Converting date to datetime format
df.loc[:,'Date'] = pd.to_datetime(df['Date'])
#Assuring that the data is sorted in asceding order according to date time
df = df.sort_values(by='Date',ascending=True)

In [14]:
#Static values should contain only one unique value, therefore investigating if that is the case here
for k, v in df[['Build_platform_target_temperature',
                 'Hot_end_temperature_target',
                 'Object_',
                 'User',
                'Total_layers',
                'State']].nunique().to_dict().items():
    print('{} = {}'.format(k,v))

Build_platform_target_temperature = 1
Hot_end_temperature_target = 1
Object_ = 1
User = 1
Total_layers = 1
State = 1


In [15]:
#Investigating unique values of "Feed_rate" and "Fan_speed"
for k, v in df[['Feed_rate',
                 'Fan_speed']].nunique().to_dict().items():
    print('{}={}'.format(k,v))

Feed_rate=10
Fan_speed=4


In [29]:
#Feed rate unit is  mm/m, therefore need to be divide by 60 for mm/s which will provide with speed in mm/s.
#Where 35 and 144 mm/s is not found in slicer as chosen paramters
df['Feed_rate'].value_counts()

4200.000     4636
3600.000     3735
1800.000     3734
3000.000     3326
1500.000     2615
1200.000     1756
10800.000    1707
2100.000      811
8640.000      734
2400.000      373
Name: Feed_rate, dtype: int64

In [27]:
#Changing fan speed to from "-", "33%"... to 0, 33,..
df.loc[df['Fan_speed'].str.contains('-'), 'Fan_speed'] = '0'
df['Fan_speed'] = df["Fan_speed"].str.replace("%", "")
#Changing "Last_layer_duration" and "Average_layer_duration"   "-" to "0"
df.loc[df['Last_layer_duration'].str.contains('-'), 'Last_layer_duration'] = '0'
df.loc[df['Average_layer_duration'].str.contains('-'), 'Average_layer_duration'] = '0'


In [19]:
#Inspecting current layer when fan speed was 0%
df.loc[df['Fan_speed'] == '0', 'Current_layer'].unique()

array(['1'], dtype=object)

In [30]:
#Renaming so all datasets contain same names of same data types
df = df.rename(columns={'Object_':'Object', 'State':'Machine_state','Hot_end_temperature_target': 'Hot_end_target_temperature'})
#Convertint values of interest into number format
df = df.astype({"Hot_end_temperature": float,
                "Build_platform_temperature": float,
                'Current_layer': int,
                'Feed_rate': float,
                'Average_layer_duration': int,
                'Last_layer_duration': int,
               'Process_time': int,
               'Feed_rate': float})

In [34]:
#Looking at unique values of fan speed and feed rate, also converted feedrate from "mm/m" to "mm/s"
dict_ = {'Feed_rate': list(df['Feed_rate'].unique()/60),
        'Fan_speed':list(df['Fan_speed'].unique()) }
dict_

{'Feed_rate': [20.0, 180.0, 35.0, 144.0, 50.0, 60.0, 30.0, 70.0, 25.0, 40.0],
 'Fan_speed': ['0', '33', '67', '100']}

In [21]:
#Storing dataframes as csv files where df1-df3 represents print1-print3 and df represents all print data
df1 = df[df['Part'] == 1]
df2 = df[df['Part'] == 2]
df3 = df[df['Part'] == 3]
df.to_csv("df.csv",   index=False)
df1.to_csv("df1.csv", index=False)
df2.to_csv("df2.csv", index=False)
df3.to_csv("df3.csv", index=False)
